In [1]:
!pip install tensorflow

In [2]:
!pip install shap

In [47]:
import pandas as pd

from tensorflow.keras import models, layers, utils, backend as k
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

import h2o
from h2o.automl import H2OAutoML

from sklearn.ensemble import RandomForestRegressor as RFR


pd.set_option('display.max_columns', None)
scaler=StandardScaler()

In [3]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sample=pd.read_csv('sample.csv')
train.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,https://www.airbnb.com/rooms/2842228,20211205213126,2021-12-06,Your little appartment on wheels,A waggon luxurous designed as a hotel room on ...,"Waterland is a beautiful countryside area, wit...",https://a0.muscache.com/pictures/cc2aa51a-3c11...,3040748,https://www.airbnb.com/users/show/3040748,Henk,2012-07-24,"Amsterdam, North Holland, The Netherlands","Living in the countryside, just near Amsterdam...",within a few hours,100%,75%,f,https://a0.muscache.com/im/users/3040748/profi...,https://a0.muscache.com/im/users/3040748/profi...,NaN,13,13,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,t,"Amsterdam, North Holland, Netherlands",Noord-Oost,NaN,52.39508,4.99186,Private room in farm stay,Private room,3,NaN,1.5 shared baths,1.0,NaN,"[""First aid kit"", ""Free parking on premises"", ...",87.0,2,1125,2.0,2.0,1125.0,1125.0,2.0,1125.0,NaN,t,0,0,0,249,2021-12-06,81,0,0,2014-08-03,2019-06-15,4.62,4.56,4.29,4.57,4.75,4.69,4.49,0363 2A6D 7C09 877D 715F,f,10,0,10,0,0.91
1,1,https://www.airbnb.com/rooms/41236669,20211205213126,2021-12-06,Spacious 4p city center apartment (2 bedrooms!),Are you looking for a perfect place to stay in...,Our home is located in the Plantagebuurt. A un...,https://a0.muscache.com/pictures/5f33bf5a-399c...,34773624,https://www.airbnb.com/users/show/34773624,Julie,2015-06-01,"Amsterdam, Noord-Holland, Netherlands",NaN,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/a5da3...,https://a0.muscache.com/im/pictures/user/a5da3...,Weesperbuurt en Plantage,0,0,"['email', 'phone', 'reviews']",t,f,"Amsterdam, Noord-Holland, Netherlands",Centrum-Oost,NaN,52.36371,4.90745,Entire rental unit,Entire home/apt,4,NaN,1 bath,2.0,2.0,"[""Hot water kettle"", ""Pocket wifi"", ""Stove"", ""...",250.0,2,1125,2.0,2.0,1125.0,1125.0,2.0,1125.0,NaN,t,3,3,3,3,2021-12-06,9,4,1,2020-01-26,2021-09-05,5.00,5.00,5.00,4.89,5.00,5.00,4.89,0363 BD0E 9934 ACCA D3DE,f,1,1,0,0,0.40
2,2,https://www.airbnb.com/rooms/27240740,20211205213126,2021-12-06,SWEETS hotel Theophile de Bockbrug,Comfortable bridge house on the doorstep of Am...,"The neighbourhood has a quiet, village-like fe...",https://a0.muscache.com/pictures/b1145dd1-e485...,203731852,https://www.airbnb.com/users/show/203731852,SWEETS Hotel,2018-07-20,"Wassenaar, South Holland, Netherlands",SWEETS hotel is a one-of-a-kind hotel located ...,within an hour,100%,97%,t,https://a0.muscache.com/im/pictures/user/492cd...,https://a0.muscache.com/im/pictures/user/492cd...,Hoofddorppleinbuurt,20,20,"['email', 'phone']",t,t,"Amsterdam, Noord-Holland, Netherlands",Zuid,NaN,52.35482,4.85329,Room in serviced apartment,Hotel room,2,NaN,1 private bath,1.0,1.0,"[""Air conditioning"", ""First aid kit"",

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4167 entries, 0 to 4166
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            4167 non-null   int64  
 1   listing_url                                   4167 non-null   object 
 2   scrape_id                                     4167 non-null   int64  
 3   last_scraped                                  4167 non-null   object 
 4   name                                          4167 non-null   object 
 5   description                                   4159 non-null   object 
 6   neighborhood_overview                         3004 non-null   object 
 7   picture_url                                   4167 non-null   object 
 8   host_id                                       4167 non-null   int64  
 9   host_url                                      4167 non-null   o

In [4]:
columnas=['property_type',
 'neighbourhood_cleansed',
 'host_location',
 'availability_30',
 'accommodates',

 'bedrooms',
 'availability_365',
 'room_type', 'reviews_per_month','review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',]

entreno=train[columnas]
testeo=test[columnas]
entreno.shape, testeo.shape

((4167, 17), (1389, 17))

In [5]:
junto=pd.concat([entreno, testeo])
junto.tail()

,property_type,neighbourhood_cleansed,host_location,availability_30,accommodates,bedrooms,availability_365,room_type,reviews_per_month,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
1384,Private room in guest suite,Oud-Noord,NL,0,2,1.0,33,Private room,0.34,4.93,4.93,4.87,4.93,4.93,4.33,4.67,f
1385,Entire rental unit,Oostelijk Havengebied - Indische Buurt,"Amsterdam, North Holland, Netherlands",0,3,2.0,0,Entire home/apt,0.46,4.63,4.83,4.29,4.83,4.78,4.57,4.32,f
1386,Entire townhouse,De Baarsjes - Oud-West,"Amsterdam, Noord-Holland, The Netherlands",0,4,2.0,0,Entire home/apt,0.21,4.93,5.00,4.86,4.93,5.00,4.71,4.71,f
1387,Entire residential home,Bos en Lommer,"Amsterdam, North Holland, Netherlands",0,2,1.0,64,Entire home/apt,1.41,5.00,5.00,5.00,5.00,5.00,5.00,4.67,f
1388,Private room in townhouse,IJburg - Zeeburgereiland,"Amsterdam, North Holland, Netherlands",12,4,NaN,267,Private room,2.21,4.84,4.88,4.82,4.95,4.99,4.71,4.63,t


In [6]:
locatio={'NL':'Netherlands',
         'Netherlands':'Netherlands',
         'US':'US',
         'RS':'Russia',
         'Spain':'Spain',
         'Ireland':'Ireland',
         'States':'US',
         'Switzerland':'Switzerland',
         'Canada':'Canada',
         'Romania':'Romania',
         'Australia':'Australia',
         'Africa':'Unknown',
         'Emirates':'Emirates',
         'Belgium':'Belgium',
         'Lithuania':'Lithuania',
         'China':'China',
         'Singapore':'China',
         'Denmark':'Denmark',
          'GB':'UK',
          'Kingdom':'UK',
         'ES':'Spain',
         'DE':'Germany',
         'BR':'Brazil',
         'London':'UK',
         'Nederland':'Netherlands',
         'FR':'France',
         'IT':'Italy'}

In [7]:
junto.host_location=junto.host_location.map(locatio)
junto.host_location.fillna('Unknown', inplace=True)


junto_dum=pd.get_dummies(junto, columns=['property_type', 'neighbourhood_cleansed','host_location',
        'room_type', 'instant_bookable'], drop_first=True)


fit_junto=pd.DataFrame(scaler.fit_transform(junto_dum))

In [8]:
fit_junto.shape

(5556, 104)

In [9]:
5556-4167

1389

In [43]:
entreno_t=fit_junto[:4167]
testeo_t=fit_junto[-1389:]

In [45]:
entreno_t['price']=train.price


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [59]:
k=junto_dum.columns
k

Index(['availability_30', 'accommodates', 'bedrooms', 'availability_365',
       'reviews_per_month', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication',
       ...
       'host_location_Netherlands', 'host_location_Russia',
       'host_location_Spain', 'host_location_UK', 'host_location_US',
       'host_location_Unknown', 'room_type_Hotel room',
       'room_type_Private room', 'room_type_Shared room',
       'instant_bookable_t'],
      dtype='object', length=104)

**TEST**

In [44]:
testeo_t.columns=junto_dum.columns
entreno_t.columns=junto_dum.columns

In [13]:
h2o.init()
h2train=h2o.H2OFrame(entreno_t)
h2test=h2o.H2OFrame(testeo_t)


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,22 hours 20 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,25 days
H2O_cluster_name:,H2O_from_python_alima_qo25af
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.627 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [14]:
automl=H2OAutoML(max_models=50,
                 seed=42,   # random_state
                 max_runtime_secs=300,
                 sort_metric='RMSE')

In [46]:
X=[c for c in entreno_t.columns if c!='price']
y='price'

In [16]:
automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |█
15:27:16.376: AutoML: XGBoost is not available; skipping it.
15:27:16.382: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]
15:27:17.44: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

██
15:27:21.532: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

████
15:27:34.178: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

██
15:27:44.430: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

██
15:27:59.247: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

███
15:28:15.295: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

███
15:28:34.463: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

█
15:28:38.301: _train param, Dropping bad and constant columns: [45, 13, 57, 36, 47]

█████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
M

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,46.0,46.0,107208.0,9.0,9.0,9.0,44.0,272.0,181.26086




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 1858.0486960485782
RMSE: 43.10508898086835
MAE: 29.471442499977808
RMSLE: 0.268697908577984
Mean Residual Deviance: 1858.0486960485782

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 9910.522387545703
RMSE: 99.55160665476828
MAE: 53.949934849377215
RMSLE: 0.4341081657080728
Mean Residual Deviance: 9910.522387545703

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,53.724770,1.917007,53.797710,53.989697,54.968563,55.365635,50.502240
1,mean_residual_deviance,10394.057000,2024.696800,10963.333000,9155.078000,12660.529000,11611.446000,7579.894000
2,mse,10394.057000,2024.696800,10963.333000,9155.078000,12660.529000,11611.446000,7579.894000
3,r2,0.441414,0.373082,0.341837,0.847432,0.046427,0.818900,0.152475
4,residual_deviance,10394.057000,2024.696800,10963.333000,9155.078000,12660.529000,11611.446000,7579.894000
5,rmse,101.545230,10.162620,104.705940,95.682175,112.519020,107.756420,87.062584
6,rmsle,0.429478,0.014186,0.425001,0.425013,0.449406,0.436384,0.411585



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-09 15:30:51,2 min 11.520 sec,0.0,180.636876,79.357361,32629.681119
1,,2022-05-09 15:30:51,2 min 11.664 sec,5.0,135.883731,62.785128,18464.388386
2,,2022-05-09 15:30:51,2 min 11.783 sec,10.0,106.168823,52.576564,11271.818971
3,,2022-05-09 15:30:52,2 min 11.895 sec,15.0,82.787823,45.915305,6853.823593
4,,2022-05-09 15:30:52,2 min 12.056 sec,20.0,70.813319,40.996384,5014.526099
5,,2022-05-09 15:30:52,2 min 12.176 sec,25.0,61.786602,37.600986,3817.584208
6,,2022-05-09 15:30:52,2 min 12.313 sec,30.0,54.970641,34.796983,3021.771402
7,,2022-05-09 15:30:52,2 min 12.431 sec,35.0,49.494727,32.853648,2449.728017
8,,2022-05-09 15:30:52,2 min 12.525 sec,40.0,46.746071,31.443295,2185.195115
9,,2022-05-09 15:30:52,2 min 12.642 sec,45.0,43.575549,29.774916,1898.828467



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,94,52872308.0,1.000000,0.105116
1,2,50411096.0,0.953450,0.100222
2,72,36670984.0,0.693576,0.072906
3,3,35776344.0,0.676656,0.071127
4,56,33371910.0,0.631179,0.066347
5,1,28793476.0,0.544585,0.057244
6,8,23663318.0,0.447556,0.047045
7,6,22340456.0,0.422536,0.044415
8,101,21381864.0,0.404406,0.042509
9,70,16634791.0,0.314622,0.033072



See the whole table with table.as_data_frame()


In [17]:
print('[INFO] Leader board:')

leader_board=automl.leaderboard

leader_board.head()

[INFO] Leader board:


model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_6_20220509_152716_model_17,99.5516,9910.52,53.9499,0.434108,9910.52
GBM_grid_1_AutoML_6_20220509_152716_model_3,100.498,10099.8,55.8237,nan,10099.8
GBM_grid_1_AutoML_6_20220509_152716_model_5,101.059,10212.9,54.3693,nan,10212.9
GBM_grid_1_AutoML_6_20220509_152716_model_8,102.453,10496.5,54.941,nan,10496.5
GBM_5_AutoML_6_20220509_152716,104.337,10886.3,52.3185,0.419723,10886.3
GBM_3_AutoML_6_20220509_152716,104.67,10955.8,54.9036,nan,10955.8
GBM_4_AutoML_6_20220509_152716,104.729,10968.2,54.767,nan,10968.2
GBM_grid_1_AutoML_6_20220509_152716_model_15,105.434,11116.3,54.9524,nan,11116.3
DRF_1_AutoML_6_20220509_152716,106.992,11447.4,52.0591,0.41533,11447.4
GBM_grid_1_AutoML_6_20220509_152716_model_1,108.926,11864.8,53.699,nan,11864.8


In [18]:
y_pred=automl.leader.predict(h2test)
automl.leader.summary()
pred= h2o.as_list(y_pred)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [19]:
sample['price']=pred

In [20]:
sample.tail()

,price,id
1384,82.351164,1384
1385,123.991542,1385
1386,195.873223,1386
1387,142.367620,1387
1388,81.984694,1388


In [21]:
pred.shape

(1389, 1)

In [23]:
sample.to_csv('lunes.csv', index=False)

In [24]:
k=pd.read_csv('lunes.csv')

In [25]:
k.head()

,price,id
0,87.155864,0
1,121.670039,1
2,119.715487,2
3,99.517998,3
4,136.249520,4


**Random Forest**

In [48]:
bosque=RFR()

In [61]:
entreno_b=entreno_t.dropna(axis=0)
Xb=entreno_b.drop('price', axis=1)


yb=entreno_b.price

In [63]:
bosque.fit(Xb,yb)

RandomForestRegressor()

In [71]:
testeo_t.head()

,availability_30,accommodates,bedrooms,availability_365,reviews_per_month,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,property_type_Boat,property_type_Bus,property_type_Camper/RV,property_type_Casa particular,property_type_Cave,property_type_Entire cabin,property_type_Entire chalet,property_type_Entire condominium (condo),property_type_Entire cottage,property_type_Entire guest suite,property_type_Entire guesthouse,property_type_Entire home/apt,property_type_Entire loft,property_type_Entire place,property_type_Entire rental unit,property_type_Entire residential home,property_type_Entire serviced apartment,property_type_Entire townhouse,property_type_Entire vacation home,property_type_Entire villa,property_type_Houseboat,property_type_Private room,property_type_Private room in bed and breakfast,property_type_Private room in boat,property_type_Private room in bungalow,property_type_Private room in cabin,property_type_Private room in casa particular,property_type_Private room in condominium (condo),property_type_Private room in farm stay,property_type_Private room in guest suite,property_type_Private room in guesthouse,property_type_Private room in hostel,property_type_Private room in houseboat,property_type_Private room in island,property_type_Private room in loft,property_type_Private room in nature lodge,property_type_Private room in rental unit,property_type_Private room in residential home,property_type_Private room in serviced apartment,property_type_Private room in tiny house,property_type_Private room in townhouse,property_type_Private room in villa,property_type_Room in aparthotel,property_type_Room in bed and breakfast,property_type_Room in boutique hotel,property_type_Room in casa particular,property_type_Room in hostel,property_type_Room in hotel,property_type_Room in serviced apartment,property_type_Shared room in bed and breakfast,property_type_Shared room in boat,property_type_Shared room in hostel,property_type_Shared room in houseboat,property_type_Shared room in rental unit,property_type_Shared room in residential home,property_type_Tiny house,property_type_Tower,neighbourhood_cleansed_Bijlmer-Oost,neighbourhood_cleansed_Bos en Lommer,neighbourhood_cleansed_Buitenveldert - Zuidas,neighbourhood_cleansed_Centrum-Oost,neighbourhood_cleansed_Centrum-West,neighbourhood_cleansed_De Aker - Nieuw Sloten,neighbourhood_cleansed_De Baarsjes - Oud-West,neighbourhood_cleansed_De Pijp - Rivierenbuurt,neighbourhood_cleansed_Gaasperdam - Driemond,neighbourhood_cleansed_Geuzenveld - Slotermeer,neighbourhood_cleansed_IJburg - Zeeburgereiland,neighbourhood_cleansed_Noord-Oost,neighbourhood_cleansed_Noord-West,neighbourhood_cleansed_Oostelijk Havengebied - Indische Buurt,neighbourhood_cleansed_Osdorp,neighbourhood_cleansed_Oud-Noord,neighbourhood_cleansed_Oud-Oost,neighbourhood_cleansed_Slotervaart,neighbourhood_cleansed_Watergraafsmeer,neighbourhood_cleansed_Westerpark,neighbourhood_cleansed_Zuid,host_location_Brazil,host_location_France,host_location_Germany,host_location_Italy,host_location_Netherlands,host_location_Russia,host_location_Spain,host_location_UK,host_location_US,host_location_Unknown,room_type_Hotel room,room_type_Private room,room_type_Shared room,instant_bookable_t
4167,2.205119,-1.338502,-0.63484,-0.134244,0.227385,0.597135,0.639331,0.771467,0.514452,0.475117,0.879330,1.170131,-0.119326,-0.013417,-0.023243,-0.013417,-0.013417,-0.023243,-0.018976,-0.227322,-0.026841,-0.067231,-0.048428,-0.013417,-0.168843,-0.030012,-0.852372,-0.259683,-0.152947,-0.175493,-0.026841,-0.058579,-0.13607,-0.035517,-0.229068,-0.115386,-0.013417,-0.018976,-0.037973,-0.107094,-0.035517,-0.128318,-0.037973,-0.05203,-0.127591,-0.013417,-0.089345,-0.013417,2.901198,-0.155395,-0.035517,-0.023243,-0.128318,-0.03288,-0.068568,-0.07117,-0.171642,-0.013417,-0.060106,-0.131894,-0.053741,-0.018976,-0.013417,-0.040280,-0.018976,-0.0

In [72]:
testeo_b=testeo_t.fillna(0)

In [73]:
y_pred_bosque=bosque.predict(testeo_b)

In [74]:
y_pred_bosque

array([ 98.6 , 124.57, 113.73, ..., 211.32, 141.83, 130.5 ])

In [75]:
sample['price']=y_pred_bosque
sample.head()

,price,id
0,98.60,0
1,124.57,1
2,113.73,2
3,92.80,3
4,119.28,4


In [76]:
sample.to_csv('lunes_bosque.csv', index=False)

In [77]:
X_train, X_test, y_train, y_test = tts(Xb, yb, 
                                                    test_size=0.2, train_size=0.8,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2838, 104), (710, 104), (2838,), (710,))

In [78]:
bosque.fit(X_train,y_train)

RandomForestRegressor()

In [79]:
y_prueba=bosque.predict(X_test)

In [80]:
from sklearn.metrics import mean_squared_error as mse

In [81]:
mse(y_prueba,y_test, squared=False)


77.03865158221316

In [90]:
from sklearn.metrics import r2_score as r2

r2(y_prueba,y_test)

0.051657834782298684

**Prueba Viernes, Investigación**

In [83]:
viernes=pd.read_csv('Data Viernes/pred_viernes.csv')

In [84]:
viernes.head()

,price,id
0,89.098731,0
1,55.408042,1
2,126.185660,2
3,130.313015,3
4,126.443382,4


In [85]:
mse(y_pred_bosque,viernes.price, squared=False)

76.90230690450103

In [86]:
mse(y_pred_bosque,pred, squared=False)

70.85402709875545

In [87]:
mse(pred,viernes.price, squared=False)

71.92272484965227

In [88]:
pred_kagle=pd.read_csv('pred.csv')
mse(pred_kagle.price,viernes.price, squared=False)

60.83078251416547

In [89]:
mse(pred_kagle.price,y_pred_bosque, squared=False)

71.52410476427454